# Как посчитать среднюю длину сессии на пользователя

### Условие:
Пусть есть таблица с данными действий пользователей со столбцами `user_id` и `event_time`. 

Действия пользователей естественно поделить на сессии - последовательности событий, в рамках
которых между соседними по времени событиями (предположим) промежуток не более 10 минут. Т.е. длина всей сессии может быть гораздо больше 10 минут, но между каждыми последовательными
событиями не должно быть более 10 минут.
### Задача:
Посчитать среднюю длину сессии на пользователя.

Получим таблицу с данными:

In [1]:
import pandas as pd

In [2]:
data = [['1','15:56:53'], 
        ['1','15:48:43'], 
        ['2','14:25:36'],
        ['1','16:04:21'],
        ['1','17:17:05'],
        ['3','16:25:53'],
        ['1','17:25:11'], 
        ['3','16:22:14'],
        ['2','15:48:17'],  
        ['4', '12:06:23'],
        ['3','16:35:47'],
        ['4', '12:15:14'],
        ['5', '18:06:54'],
        ['4', '12:23:08'],
        ['5', '18:04:11'],
        ['5', '16:55:08'],
        ['4', '12:02:19']]

df = pd.DataFrame(data=data, columns=['user_id', 'event_time'])
display(df)

,user_id,event_time
0,1,15:56:53
1,1,15:48:43
2,2,14:25:36
3,1,16:04:21
4,1,17:17:05
5,3,16:25:53
6,1,17:25:11
7,3,16:22:14
8,2,15:48:17
9,4,12:06:23


Записи перемешаны. Преобразуем столбец `event_time` к типу datetime и отсортируем записи по пользователю и времени события:

In [3]:
df['event_time'] = pd.to_datetime(df['event_time'])
df = df.sort_values(by=['user_id', 'event_time']).reset_index(drop=True)
display(df)

,user_id,event_time
0,1,2020-11-30 15:48:43
1,1,2020-11-30 15:56:53
2,1,2020-11-30 16:04:21
3,1,2020-11-30 17:17:05
4,1,2020-11-30 17:25:11
5,2,2020-11-30 14:25:36
6,2,2020-11-30 15:48:17
7,3,2020-11-30 16:22:14
8,3,2020-11-30 16:25:53
9,3,2020-11-30 16:35:47


Ддя того, чтобы получить длину сессии, сгруппируем пользователей по времени события и сдвинем все записи на одну строку вверх, затем вычтем из столбца `event_time` новый столбец:

In [4]:
df['lag']= df.groupby('user_id')['event_time'].shift()
df['session_length'] = (df['event_time'] - df['lag']) / pd.Timedelta('1M')
display(df)

,user_id,event_time,lag,session_length
0,1,2020-11-30 15:48:43,NaT,NaN
1,1,2020-11-30 15:56:53,2020-11-30 15:48:43,8.166667
2,1,2020-11-30 16:04:21,2020-11-30 15:56:53,7.466667
3,1,2020-11-30 17:17:05,2020-11-30 16:04:21,72.733333
4,1,2020-11-30 17:25:11,2020-11-30 17:17:05,8.100000
5,2,2020-11-30 14:25:36,NaT,NaN
6,2,2020-11-30 15:48:17,2020-11-30 14:25:36,82.683333
7,3,2020-11-30 16:22:14,NaT,NaN
8,3,2020-11-30 16:25:53,2020-11-30 16:22:14,3.650000
9,3,2020-11-30 16:35:47,2020-11-30 16:25:53,9.900000


Чтобы не учитывать записи длиной более 10 минут при расчете средней сессии, удалим их из данных. Бонусом добавим число сессий не более 10 минут и номер сессии:

In [5]:
df.loc[df['session_length'] > 10, 'session_length'] = None
df['sessions_count_by_user'] = df.groupby('user_id')['session_length'].transform('count')
df['session_num']=(df.groupby('user_id')['session_length']
                   .expanding().count().reset_index().set_index('level_1').drop('user_id', axis=1).astype(int))
display(df)

,user_id,event_time,lag,session_length,sessions_count_by_user,session_num
0,1,2020-11-30 15:48:43,NaT,NaN,3,0
1,1,2020-11-30 15:56:53,2020-11-30 15:48:43,8.166667,3,1
2,1,2020-11-30 16:04:21,2020-11-30 15:56:53,7.466667,3,2
3,1,2020-11-30 17:17:05,2020-11-30 16:04:21,NaN,3,2
4,1,2020-11-30 17:25:11,2020-11-30 17:17:05,8.100000,3,3
5,2,2020-11-30 14:25:36,NaT,NaN,0,0
6,2,2020-11-30 15:48:17,2020-11-30 14:25:36,NaN,0,0
7,3,2020-11-30 16:22:14,NaT,NaN,2,0
8,3,2020-11-30 16:25:53,2020-11-30 16:22:14,3.650000,2,1
9,3,2020-11-30 16:35:47,2020-11-30 16:25:53,9.900000,2,2


Расчет средней длины сессии на пользователя:

In [6]:
mean_session_length = (df.groupby('user_id')['session_length'].mean()
                       .sort_values(ascending=False).to_frame('mean_session_length'))
display(mean_session_length)

,mean_session_length
user_id,
1,7.911111
4,6.938889
3,6.775000
5,2.716667
2,NaN
